In [1]:
# import files

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
from datetime import date, timedelta
import pandas as pd

import time

In [2]:
# Global variables
search_field_term_job = "data scientist"
search_field_term_location = "New York, NY"


In [3]:
# define driver
driver = webdriver.Chrome()

# open url
driver.get("https://www.simplyhired.com/")

# wait for the page to load
driver.implicitly_wait(5)


# find and click on search field location
search_field = driver.find_element(By.XPATH, '//*[@id="field-:R5bakt9fbqm:"]')
search_field.click()

# clear search field
#search_field.clear()
search_field.send_keys(Keys.CONTROL + "a")
search_field.send_keys(Keys.DELETE)

# enter search term job
search_field.send_keys(search_field_term_location)




# find and click on search field job
search_field = driver.find_element(By.XPATH, '//*[@id="field-:R3bakt9fbqm:"]')
search_field.click()

# wait for the page to load
driver.implicitly_wait(5)

# enter search term job
search_field.send_keys(search_field_term_job)

element = driver.switch_to.active_element
element.send_keys(Keys.ESCAPE)

# wait for the page to load
driver.implicitly_wait(5)



# submit form
# search_field.submit()
search_field.send_keys(Keys.ENTER)


# set for past 24 hours
driver.find_element(By.XPATH, '//*[@id="menu-button-:rl:"]').click()
time.sleep(2)
driver.find_elements(By.XPATH, '//*[@id="menu-list-:rl:"]//button')[1].click()



In [4]:
def getElemt(xpath, wait):
    wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_element(By.XPATH, xpath)

In [5]:
def pullJobData():

    time.sleep(3)
    # get number of jobs
    webElementJobList = driver.find_element(By.XPATH, '//*[@id="job-list"]')
    num_rows = len(webElementJobList.find_elements(By.XPATH, '//li[@class="css-0"]'))
    wait = WebDriverWait(driver, timeout=5)


    # if link not found, skip to next job
    skip = False

    # list of jobs
    job_list = []

    for i in range(1, num_rows+1):

        job_desc = {
                    "jobTitle" : '',
                    'company':'',
                    'location':'',
                    'salary':'',
                    'date pulled': date.today(),
                    'link':'',
                    'description':'',
                    'qualifications':''
                
                    }

        # Get job title
        jobTitleXpath = f'//li[@class="css-0"][{i}]//a'
        for tries in range(2):
            try:
                job_desc["jobTitle"] = getElemt(jobTitleXpath, wait = wait).text
                break
            except Exception as e:
                print('job title not found')

        # Get link
        linkXpath =  f'//li[@class="css-0"][{i}]//a'
        for tries in range(2):
            try:
                job_desc['link'] = getElemt(linkXpath, wait = wait).get_attribute('href')

                break
            except Exception as e:
                print('link not found')


        # click on job
        linkXpath =  f'//li[@class="css-0"][{i}]//a'
        for tries in range(2):
            try:
                getElemt(linkXpath, wait = wait).click()
                break
            except:
                print('link not clickable')

                # skip to next job
                skip = True

        if skip:
            continue

        time.sleep(3)
        # Get description
        descriptionXpath = '//*[@data-testid="viewJobBodyJobFullDescriptionContent"]'
        for tries in range(2):
            try:
                job_desc['description'] = getElemt(descriptionXpath, wait = wait).text
                break
            except:
                print('description not found')

        # Get company and location
        xpath = '//*[@data-testid="viewJobHeadingContainer"]//span[@data-testid="detailText"]'
        for tries in range(2):
            try:
                wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
                job_desc['company'], job_desc['location']  = [ele.text for ele in driver.\
                        find_elements(By.XPATH, xpath)]
                break
            except:
                print('company and location not found')

        # Get salary
        salaryXpath = '//*[@data-testid="viewJobBodyJobCompensation"]'
        for tries in range(2):
            try:
                job_desc['salary']  = getElemt(salaryXpath, wait = wait).text
                break
            except:
                print('Salary not found')
        

        # Get qualification
        xpath = '//*[@id="__next"]/div/main/div/div[2]/div/div/div[2]/div/aside/div/div[1]/div/div[3]/div/div/ul//li'
        for tries in range(2):
            try:
                qualifications = [detail.text for detail in driver.find_elements(By.XPATH, xpath)]
                job_desc['qualifications'] = "||".join(qualifications)
                break
            except:
                print('qualifications not found')
                
        job_list.append(job_desc)
    
    return job_list

In [6]:
jobs = []

for _ in range(2):
    print(_)
    jobs.extend(pullJobData())

    try:
        next_page = driver.find_element(By.XPATH, '//*[@id="__next"]/div/main//nav//span/following-sibling::a').click()
        next_page.click()
    except:
        print('No more pages')
        break

    # let page load(driver.implicitly_wait does not work
    time.sleep(3)




0
No more pages


In [7]:
jobs = pd.DataFrame(jobs) #, columns = ['jobTitle', 'company', 'location', 'salary', 'date posted', 'link', 'describtion', 'qualifications'])
jobs.head()

,jobTitle,company,location,salary,date pulled,link,description,qualifications
0,"Manager, Data Science",Syneos Health - USA,United States,"$85,500 - $175,700 a year",2024-02-12,https://www.simplyhired.com/job/an1HJilbspAoLE...,Description\n\nSyneos Health® is a leading ful...,Doctoral degree||5 years||R||Master's degree||...
1,Senior Applied Scientist,Amazon.com,"Newark, NJ",Estimated: $135K - $171K a year,2024-02-12,https://www.simplyhired.com/job/8I68i5URWijeWN...,"At Audible, we believe stories have the power ...",Law||Doctoral degree||5 years||Big data||Spark...
2,Data Engineer/Sr Data Engineer,DNV,"New York, NY","$105,000 - $150,000 a year",2024-02-12,https://www.simplyhired.com/job/yW48VbHByhwZib...,About us\nWe are the independent expert in ass...,Azure||Spark||Git||English||SQL||Machine learn...
3,Senior Data scientist,Ocean Flex International Pvt. Ltd.,"Vauxhall, NJ",$60 - $65 an hour,2024-02-12,https://www.simplyhired.com/job/E7XLmbZUUDyQdo...,Role: Senior Gen AI/LLM Data scientist\nLocati...,Law||Autoencoder||SAS||Master's degree||Machin...


In [8]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   jobTitle        4 non-null      object
 1   company         4 non-null      object
 2   location        4 non-null      object
 3   salary          4 non-null      object
 4   date pulled     4 non-null      object
 5   link            4 non-null      object
 6   description     4 non-null      object
 7   qualifications  4 non-null      object
dtypes: object(8)
memory usage: 384.0+ bytes


In [9]:
jobs.to_csv(f'data/simplyHiredScrape_{date.today()}.csv', index = False )

In [10]:
driver.close()